# TensorFlow Overview

## Programs as compute graphs

In [ ]:
import tensorflow as tf
a = tf.ones([100, 10], name='a')
b = tf.ones([100, 10], name='b')
c = a + b
print(c) # Tensor("add:0", shape=(100, 10), dtype=float32)

## Executing programs with a session

In [ ]:
with tf.Session() as sess:
    value_of_c = sess.run(c)
print(value_of_c) # [[2. 2. 2. ...], [2. 2. 2. ...], ...]

In [ ]:
with tf.Session() as sess:
    value_of_a , value_of_c = sess.run([a, c])
print(value_of_a) # [[2. 2. 2. ...], [2. 2. 2. ...], ...]
print(value_of_c) # [[2. 2. 2. ...], [2. 2. 2. ...], ...]

## Stateful programs using variables

In [ ]:
a = tf.get_variable(name='variable_a',
                    shape=[],
                    dtype=tf.float32 ,
                    initializer=tf.constant_initializer(5))
print(a)  # <tf.Variable 'variable_a:0' shape=() dtype=float32_ref>

In [ ]:
initialize_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(initialize_op) 
    print(sess.run(a)) # 5.0

In [ ]:
assign_op = tf.assign(a, 10)
with tf.Session() as sess: 
    sess.run(initialize_op) 
    print(sess.run(a)) # 5.0
    sess.run(assign_op) 
    print(sess.run(a)) # 10.0

## Automatic differentiation

In [ ]:
loss = a ** 2
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1) 
minimize_op = optimizer.minimize(loss)
with tf.Session() as sess: 
    sess.run(initialize_op)
    print(sess.run(a)) # 5.0 
    sess.run(minimize_op) 
    print(sess.run(a)) # 4.0
    sess.run(minimize_op) 
    print(sess.run(a)) # 3.2

## Execution order and dependencies

In [ ]:
assign_op = a.assign(42) 
b = tf.constant(1.)
c = a + b
with tf.Session() as sess:
    sess.run(initialize_op)
    value, _ = sess.run([c, assign_op]) 
    print(value) # Sometimes 6.0, sometimes 43.0

In [ ]:
assign_op = a.assign(42) 
b = tf.constant(1.)

with tf.control_dependencies([assign_op]):
    c = a + b
    
with tf.Session() as sess:
    sess.run(initialize_op)
    value = sess.run(c)  # Update a and then compute c
    print(value) # Always 43.0
    print(sess.run(a)) # 42

## For-loops inside the graph

In [ ]:
def loop_body(last_output, current_input):
    # tf.scan() supports nested tuples, lists, and dict objects as
    # inputs and outputs. In this example, the input sequence is a
    # single tensor and the output sequence is a tuple of two tensors.
    value = last_output[0] + current_input
    current_output = (value, value * 2)
    return current_output

input_sequence = tf.constant([1, 2, 3, 4, 5])
# The initializer is passed to the loop body as `last_output`
# parameter at the first iteration. It must have the same type and
# shape as the output of the loop body: a tuple of two scalar
# integers in our case.
initializer = (tf.constant(0), tf.constant(0))
output_sequence = tf.scan(loop_body, input_sequence, initializer)

print(tf.Session().run(output_sequence))
# ([1, 3, 6, 10, 15], [2, 6, 12, 20, 30])